In [11]:
import pickle
import shap
import os
import pandas as pd
from Fast_BEX import optimized_bex
# from Bex import bex
import xgboost as xgb
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
import numpy as np

In [6]:
# df = pd.read_csv("../CSVs and Output Files/HN_P2C_20231207.csv")
df = pd.read_csv("/Users/mohamedhassan/Library/CloudStorage/OneDrive-UniversityofCalifornia,Davis/HN Model Development/Data/HN_P2C_20231207.csv")
cwd = os.getcwd()

/var/folders/vd/fl4jxzz5645dmfqvhypcn1wm0000gn/T/ipykernel_2237/2055828598.py:2: DtypeWarning: Columns (10,12,17,18,19,20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mohamedhassan/Library/CloudStorage/OneDrive-UniversityofCalifornia,Davis/HN Model Development/Data/HN_P2C_20231207.csv")


In [4]:
df['Case'].unique()

array([  2,   4,   5,   6,   7,   8,  10,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179])

In [7]:
df = df[df['ScanContext'] == 'In Vivo']

# %% SNR and lifetime filtering
index = df[((df['snr_ch1'] < 30) & (df['Instrument'] == 'V4')) |
           ((df['snr_ch2'] < 30) & (df['Instrument'] == 'V4')) |
           ((df['snr_ch3'] < 30) & (df['Instrument'] == 'V4'))].index
df.drop(index, inplace=True)
index = df[((df['snr_ch1'] < 50) & (df['Instrument'] == 'FLImBrush')) |
           ((df['snr_ch2'] < 50) & (df['Instrument'] == 'FLImBrush')) |
           ((df['snr_ch3'] < 50) & (df['Instrument'] == 'FLImBrush'))].index
df.drop(index, inplace=True)

# R_data((R_data.lifet_avg_ch1 > 10 | R_data.lifet_avg_ch1 < 2),:) = [];
index = df[((df['lifet_avg_ch1'] > 16) | (df['lifet_avg_ch1'] < 1)) |
           ((df['lifet_avg_ch2'] > 16) | (df['lifet_avg_ch2'] < 1)) |
           ((df['lifet_avg_ch3'] > 16) | (df['lifet_avg_ch3'] < 1))].index
df.drop(index, inplace=True)

# %% For our current analysis we remove all V4 data and no channel 4 data
index = df[df['Instrument'] == 'FLImBrush'].index
df.drop(index, inplace=True)

In the following code, I changed (in line 3)
`selected_columns = df.columns[df.columns.str.contains(pattern)]` to
`selected_columns = df.columns[df.columns.str.match(pattern)]`

and same edits in line 28 - any neg effects?


In [8]:
# Extract columns based on specific patterns
pattern = r'(Case|Run|snr_|lifet_avg_|spec_int_|int_ratio_|Laguerre_coeffs_|Phasor_GH|Phasor_SH|WLI_)'
selected_columns = df.columns[df.columns.str.contains(pattern)]

# Add the "Label" column, which doesn't fit the patterns
selected_columns = selected_columns.insert(len(selected_columns), 'Label')

subset_df = df[selected_columns]
subset_df = subset_df.dropna()

subset_df = subset_df[subset_df['Label'] != 0]
# Replace values in the 'Label' column
subset_df['Label'] = subset_df['Label'].replace([1, 2, 18, 20, 25, 27, 29, 36, 37], 0)
subset_df['Label'] = subset_df['Label'].replace([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 22, 24, 28, 30, 33], 1)

# Remove rows where 'Label' is not 0 or 1
subset_df = subset_df[subset_df['Label'].isin([0, 1])]

/var/folders/vd/fl4jxzz5645dmfqvhypcn1wm0000gn/T/ipykernel_2237/3490899123.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  selected_columns = df.columns[df.columns.str.contains(pattern)]


In [9]:


bex_df = optimized_bex(subset_df) #excludes weak labels
# bex_df = bex(subset_df)
f = 1

# Bex(subset_df, l_k=48, f_k1=6, f_k2=47)
# %% For our current analysis we remove all V4 data and no channel 4 data
from scipy.spatial.distance import pdist

pattern = r'(Case|Run|snr_|lifet_avg_|spec_int_|int_ratio_|Laguerre_coeffs_|WLI_)'
selected_columns = df.columns[df.columns.str.contains(pattern)]
selected_columns = selected_columns.insert(len(selected_columns), 'Label')
TrainTest_Data = bex_df[selected_columns]
TrainTest_Data.head()

/var/folders/vd/fl4jxzz5645dmfqvhypcn1wm0000gn/T/ipykernel_2237/1873497049.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  selected_columns = df.columns[df.columns.str.contains(pattern)]


,Case,Run,lifet_avg_ch1,lifet_avg_ch2,lifet_avg_ch3,spec_int_ch1,spec_int_ch2,spec_int_ch3,int_ratio_ch1,int_ratio_ch2,...,Laguerre_coeffs_7_ch3,Laguerre_coeffs_8_ch3,Laguerre_coeffs_9_ch3,Laguerre_coeffs_10_ch3,Laguerre_coeffs_11_ch3,Laguerre_coeffs_12_ch3,WLI_frame_n,WLI_x_coord,WLI_y_coord,Label
0,2,10,3.677834,4.372698,4.673487,0.100970,0.105687,0.056391,0.383847,0.401777,...,0.011809,-0.012513,0.004941,-0.002295,0.001451,-0.000368,3,815,644,0
1,2,10,3.725055,4.311057,4.678097,0.085447,0.083083,0.045521,0.399188,0.388146,...,0.037673,0.003595,0.011348,0.000742,0.001862,-0.000485,4,813,642,0
2,2,10,3.809186,4.337621,4.262062,0.059230,0.059800,0.032438,0.391038,0.394803,...,0.003650,-0.004216,-0.001125,-0.000643,-0.000201,-0.000680,5,814,626,0
3,2,10,3.822145,4.295335,3.905487,0.053078,0.053375,0.029710,0.389809,0.391994,...,0.000380,-0.006706,-0.001397,-0.002114,0.000458,-0.000604,6,814,626,0
4,2,10,3.719731,4.497713,4.277077,0.059476,0.064631,0.036502,0.370316,0.402411,...,0.002631,-0.009467,-0.000464,-0.000858,0.000830,0.000674,7,818,611,0


**.pkl File Selection**

In [10]:
def load_pickle_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("Pickle files", "*.pkl")])
    
    if file_path:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        root.destroy()  # Close the tkinter root window
        return data
    else:
        print("No file selected.")
        root.destroy()  # Close the tkinter root window
        return None
    
data = load_pickle_file()
xgb_model = data['model']
dataset = data['post_processing_data']
dataset.head(250)

,Case,Run,snr_ch1,snr_ch2,snr_ch3,WLI_frame_n,WLI_x_coord,WLI_y_coord,score,gt
2181,12,2,33.607934,36.659699,34.116102,16,623,324,0.650731,1
2182,12,2,37.825374,39.156326,33.904262,17,626,322,0.152712,1
2183,12,2,37.149354,41.233085,37.948919,18,628,320,0.430317,1
2184,12,2,37.982275,41.585132,37.727075,19,628,320,0.711976,1
2185,12,2,36.610233,36.425532,31.559710,20,626,314,0.030592,1
...,...,...,...,...,...,...,...,...,...,...
2426,12,2,38.101118,42.209111,36.222743,267,608,350,0.952975,1
2427,12,2,38.659798,40.268557,34.801394,268,599,351,0.302713,1
2428,12,2,39.153304,41.559711,35.608918,269,601,351,0.687573,1
2429,12,2,38.189985,42.255496,37.374352,270,602,348,0.846065,1


In [6]:
dataset['gt'].unique()

array([1, 0])

In [24]:
Case_num = dataset['Case'].unique()
filtered_df = TrainTest_Data[TrainTest_Data['Case'].isin(Case_num)]
X_test = filtered_df.iloc[:, 2:-4]
y_test = filtered_df.iloc[:, -1]

In [1]:
perd = score<0.5

NameError: name 'score' is not defined

In [ ]:
Posclass = 1
                auc = roc_auc_score(GT, score)
                Eval.at[i, 'AUC'] = auc
                fpr, tpr, thresholds = roc_curve(GT, score, pos_label=Posclass)
                roc_metrics = {"fpr": fpr, "tpr": tpr, "thresholds": thresholds}

                opt_threshold_idx = np.argmax(tpr - fpr)
                opt_threshold = thresholds[opt_threshold_idx]
                Eval.at[i, 'OptThreshold'] = opt_threshold

            thresh = 0.5

            opt_threshold = None

            idx = score < thresh
            Pred = np.empty_like(GT, dtype=object)
            Pred[idx] = 0
            Pred[~idx] = 1
            TP = np.sum(np.logical_and(GT == 1, Pred == 1))
            TN = np.sum(np.logical_and(GT == 0, Pred == 0))
            FP = np.sum(np.logical_and(GT == 0, Pred == 1))
            FN = np.sum(np.logical_and(GT == 1, Pred == 0))

            accuracy = (TP + TN) / (TP + TN + FP + FN)
            Recall = TP / (TP + FN)
            Precision = TP / (TP + FP)
            Specificity = TN / (TN + FP)
            Sensitivity = TP / (TP + FN)

In [ ]:
ROCAUC, OptTreshold = skitlearn.rocauc_score(score,gt)

In [ ]:
X_test_TP
X_test_TN
X_test_FP
X_test_FN

**SHAP Explainer Threshold 0.5**

In [25]:
#features = TrainTest_Data

explainer = shap.Explainer(xgb_model)
shap_values = explainer.shap_values(X_test)

**Prepare SHAP Explainer & Calculate SHAP Values**

In [ ]:
import dtreeviz

In [2]:
train_indices = np.where(TrainTest_Data['Case'] != Case_num)[0]
X_train = TrainTest_Data.iloc[train_indices, 2:-4].copy()
y_train = TrainTest_Data.iloc[train_indices]['Label'].copy()

NameError: name 'np' is not defined

In [ ]:
viz_model = dtreeviz.model(xgb_model, tree_index=1, X_train = X_train, y_train = y_train, feature_names=feature_names, target_name=target_name)

v= viz_model.view()
v.save("tree_test.svg")